In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [24]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os

mainnet = 'https://api.thegraph.com/subgraphs/name/jiro-ono/sushiswap-exchange'
fantom = 'https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange'
matic = 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange'

## Matic

In [25]:
sample_transport=RequestsHTTPTransport(
    url=matic,
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

In [26]:
query = gql('''
query {
  factories(where: {liquidityUSD_gt:0}) {
    pairCount
  }
}
''')

response = client.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

852


In [28]:
df = pd.DataFrame([])
response_dict = {}
response_list = []
id_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      name
                      id
                  }
                }
                '''
        response = client.execute(gql(querystring))
        for row in response['pairs']:
            response_list.append(row['name'])
            id_list.append(row['id'])    

df = pd.DataFrame.from_dict(list(zip(response_list,id_list)))
df[['Token 0', 'Token 1']] = df[0].str.split('-', expand = True, n = 1)
df

,0,1,Token 0,Token 1
0,YELD-WMATIC,0x00fae3b6fc7bcee980c1e79a8b63a80ff6f6a0be,YELD,WMATIC
1,WMATIC-YELD,0x01748abbd18cfaf852e9149baa05346825bae2f5,WMATIC,YELD
2,WMATIC-MOON,0x01c5bb9ce9b8804ddf93e58ca823c7fbb67b2f8d,WMATIC,MOON
3,WETH-IRON,0x01cdcc99a4f9a68be18504750b219caa881841eb,WETH,IRON
4,WMATIC-DRC,0x03e526407a92bf440de42d84d31689433a66a39f,WMATIC,DRC
...,...,...,...,...
435,USDC-JPYC,0xfbae8e2d04a67c10047d83ee9b8aeffe7f6ea3f4,USDC,JPYC
436,ET-WETH,0xfbba6eaa27080321aa75efe72216989a675fcc5f,ET,WETH
437,pDAI-USDT,0xfc163252cdf492e610bbec4f4fd237dcd48136db,pDAI,USDT
438,USDC-AMIS,0xfc4c03205ef43a264f864db3378bdc0e28d6d484,USDC,AMIS


In [29]:
# interesting_tokens = ['YFI', 'WOOFY', 'CRV', 'ILV', 'WBTC', 'WMATIC', 'MATIC', 'AAVE', 'WETH', 'ETH', 'SUSHI', 'UNI', 'SNX', 'LINK', 'DAI', 'USDC', 'USDT', 'FRAX']
# interesting_tokens = ['USDC', 'USDT', 'DAI', 'FRAX', 'IRON']
interesting_tokens = ['MATIC', 'DAI', 'USDC', 'USDT', 'FRAX', 'WBTC', 'WETH', 'WMATIC']

criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.columns = ['Pair', 'id', 'Token 0', 'Token 1']
df_filtered.head()

,Pair,id,Token 0,Token 1
55,FRAX-USDT,0x264a05020b491ba1969a325b0a474a864bb0d93b,FRAX,USDT
87,USDC-WETH,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,USDC,WETH
97,DAI-USDT,0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff,DAI,USDT
125,USDC-USDT,0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001,USDC,USDT
142,WMATIC-USDT,0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e,WMATIC,USDT


In [30]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

sample_transport=RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/medariox/quicksushi',
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pair:"''' + id + '''"} orderBy: date, orderDirection: desc first: 7) {
    date
    volumeUSD
    reserveUSD
  }
}'''
    response = client.execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 16 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 2 / 16 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 3 / 16 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
 4 / 16 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
 5 / 16 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
 6 / 16 pair: 0x6ff62bfb8c12109e8000935a6de54dad83a4f39f
 7 / 16 pair: 0x7189aacc5cf6f74e87b31920dc05f2ebd6840018
 8 / 16 pair: 0x8531c4e29491fe6e5e87af6054fc20fccf0b4290
 9 / 16 pair: 0x8929d3fea77398f64448c85015633c2d6472fb29
10 / 16 pair: 0x9e20a8d3501bf96eda8e69b96dd84840058a1cb0
11 / 16 pair: 0xc2755915a85c6f6c1c0f3a86ac8c058f11caa9c9
12 / 16 pair: 0xc4e595acdd7d12fec385e5da5d43160e8a0bac0e
13 / 16 pair: 0xcd353f79d9fade311fc3119b841e1f456b54e858
14 / 16 pair: 0xcd578f016888b57f1b1e3f887f392f0159e26747
15 / 16 pair: 0xd02b870c556480491c70aaf98c297fddd93f6f5c
16 / 16 pair: 0xe62ec2e799305e0d367b0cc3ee2cda135bf89816


In [32]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['volumeUSD','reserveUSD']] = df5[['volumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s')
df5['fee'] = round(df5['volumeUSD'] * 0.003,6)
df5['1y APR for 100 invested'] = round((100/df5['reserveUSD']) * df5['fee'] * 365,3)
df5['Pair'] = '[Sushi Polygon] ' + df5['Pair']
df5.style.format({'reserveUSD': "{:0<4,.10f}"})

,id,date,reserveUSD,volumeUSD,Pair,Token 0,Token 1,fee,1y APR for 100 invested
0,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-06-19 00:00:00,"124,904,234.2952899933",7575294.356510,[Sushi Polygon] USDC-WETH,USDC,WETH,22725.883070,6.641000
1,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-06-18 00:00:00,"127,141,917.5265640020",19516694.076684,[Sushi Polygon] USDC-WETH,USDC,WETH,58550.082230,16.809000
2,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-06-17 00:00:00,"125,475,829.0796280056",38354184.026219,[Sushi Polygon] USDC-WETH,USDC,WETH,115062.552079,33.471000
3,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-06-16 00:00:00,"119,336,901.6477960050",97700921.766824,[Sushi Polygon] USDC-WETH,USDC,WETH,293102.765300,89.647000
4,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-06-15 00:00:00,"120,257,722.8929580003",25547312.174314,[Sushi Polygon] USDC-WETH,USDC,WETH,76641.936523,23.262000
5,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-06-14 00:00:00,"132,374,667.0209179968",30787846.865882,[Sushi Polygon] USDC-WETH,USDC,WETH,92363.540598,25.468000
6,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,2021-06-13 00:00:00,"137,625,927.2156920135",17334809.457765,[Sushi Polygon] USDC-WETH,USDC,WETH,52004.428373,13.792000
7,0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001,2021-06-19 00:00:00,"97,872,954.4160120040",1711192.704622,[Sushi Polygon] USDC-USDT,USDC,USDT,5133.578114,1.914000
8,0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001,2021-06-18 00:00:00,"99,817,748.5629640073",3482617.170954,[Sushi Polygon] USDC-USDT,USDC,USDT,10447.851513,3.820000
9,0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001,2021-06-17 00:00:00,"105,328,263.0244340003",11628181.958246,[Sushi Polygon] USDC-USDT,USDC,USDT,34884.545875,12.089000


In [33]:
criteria1 = df5['reserveUSD'] > 1
criteria2 = df5['date'] == df5['date'].max()

latest_results = df5.loc[(criteria1 & criteria2), :].sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'])\
                    .sort_values(by = '1y APR for 100 invested',
                                 ascending = False)\
                    .reset_index(drop = True)\
                    .drop(columns = ['id', 'Token 0', 'Token 1'])\
                    .reset_index(drop = True)[['date', 'Pair', '1y APR for 100 invested',  'reserveUSD', 'volumeUSD', 'fee']]

df_styled  = latest_results.style.format({'reserveUSD': "{:0<4,.2f}",
                             'volumeUSD': "{:0<4,.2f}",
                             'fee': "{:0<4,.2f}",
                             '1y APR for 100 invested': "{:0<4,.2f}"})\
              .set_properties(subset=["Pair"], **{'text-align': 'left'})

df_styled

,date,Pair,1y APR for 100 invested,reserveUSD,volumeUSD,fee
0,2021-06-19 00:00:00,[Sushi Polygon] WMATIC-USDC,23.40,"21,320,227.61","4,556,165.84","13,668.50"
1,2021-06-19 00:00:00,[Sushi Polygon] USDC-WETH,6.64,"124,904,234.30","7,575,294.36","22,725.88"
2,2021-06-19 00:00:00,[Sushi Polygon] WETH-DAI,5.36,"40,868,972.38","1,999,225.56","5,997.68"
3,2021-06-19 00:00:00,[Sushi Polygon] WETH-USDT,5.16,"66,089,056.35","3,116,283.06","9,348.85"
4,2021-06-19 00:00:00,[Sushi Polygon] WMATIC-WETH,4.88,"70,806,346.60","3,156,850.40","9,470.55"
5,2021-06-19 00:00:00,[Sushi Polygon] USDC-USDT,1.91,"97,872,954.42","1,711,192.70","5,133.58"
6,2021-06-19 00:00:00,[Sushi Polygon] WBTC-WETH,1.42,"167,075,296.85","2,158,652.37","6,475.96"
7,2021-06-19 00:00:00,[Sushi Polygon] FRAX-USDC,1.01,"14,066,615.10","130,391.94",391.18
8,2021-06-19 00:00:00,[Sushi Polygon] USDC-DAI,0.88,"92,960,935.14","743,207.63","2,229.62"


In [34]:
dfi.export(df_styled, 'df_styled_sushi_poly.png')
os.startfile('df_styled_sushi_poly.png')